<a href="https://colab.research.google.com/github/RaphaelPetit/Transilien-SNCF-Waiting-Time-Forecasting/blob/main/Transilien_SNCF_v0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
path1 = '/content/drive/MyDrive/Challenges Data/Transilien-SNCF/x_train_final.csv'
x_train = pd.read_csv(path1)

x_train.drop(columns=['Unnamed: 0.1', 'Unnamed: 0'], inplace=True)

path2 = '/content/drive/MyDrive/Challenges Data/Transilien-SNCF/x_test_final.csv'
x_test = pd.read_csv(path2)

x_test.drop(columns=['Unnamed: 0'], inplace=True)

path3 = '/content/drive/MyDrive/Challenges Data/Transilien-SNCF/y_train_final_j5KGWWK.csv'
y_train = pd.read_csv(path3)

y_train.drop(columns=['Unnamed: 0'], inplace=True)


In [ ]:
x_train

,train,gare,date,arret,p2q0,p3q0,p4q0,p0q2,p0q3,p0q4
0,VBXNMF,KYF,2023-04-03,8,0.0,0.0,1.0,-3.0,-1.0,-2.0
1,VBXNMF,JLR,2023-04-03,9,0.0,0.0,0.0,1.0,0.0,1.0
2,VBXNMF,EOH,2023-04-03,10,-1.0,0.0,0.0,-1.0,0.0,0.0
3,VBXNMF,VXY,2023-04-03,11,-1.0,-1.0,0.0,2.0,-2.0,0.0
4,VBXNMF,OCB,2023-04-03,12,-1.0,-1.0,-1.0,-1.0,3.0,2.0
...,...,...,...,...,...,...,...,...,...,...
667259,BFNJJK,DGG,2023-11-10,22,0.0,-1.0,-6.0,-1.0,0.0,-2.0
667260,BFNJJK,KDN,2023-11-10,23,0.0,0.0,-1.0,1.0,0.0,0.0
667261,BFNJJK,TVD,2023-11-10,24,1.0,0.0,0.0,-1.0,-3.0,-1.0
667262,BFNJJK,TXP,2023-11-10,26,2.0,2.0,1.0,0.0,0.0,1.0


In [ ]:
#mettre le bail en date time
x_train["date"] = pd.to_datetime(x_train["date"])

#creation d'une colonne weekday : 0 c'est lundi et 6 c'est dimanche
x_train["weekday"] = x_train["date"].dt.weekday

In [ ]:
features = [
    'arret',
    'p2q0', 'p3q0', 'p4q0',
    'p0q2', 'p0q3', 'p0q4',
    'weekday'
]

In [ ]:
from sklearn.metrics import make_scorer, mean_absolute_error
mae_scorer = make_scorer(mean_absolute_error, greater_is_better=False)

# Modèles sans TimeSeriesSplit

In [ ]:
x_sample = x_train.sample(50000, random_state=42)
y_sample = y_train.loc[x_sample.index]

X = x_sample[features].fillna(0)
y = y_sample["p0q0"]

# RandomForest

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score

model = RandomForestRegressor(n_estimators=20, max_depth=10, random_state=42, n_jobs=-1)
scores = cross_val_score(model, X, y, cv=5, scoring=mae_scorer)

In [ ]:
print(f"MAE moyen (5-fold CV) : {-scores.mean():.3f} ± {scores.std():.3f}")

MAE moyen (5-fold CV) : 0.852 ± 0.011


# LightGBM

In [ ]:
import lightgbm as lgb

model = lgb.LGBMRegressor(n_estimators=100, random_state=42)
scores = cross_val_score(model, X, y, cv=5, scoring=mae_scorer)

print(f"LightGBM MAE : {-scores.mean():.3f} ± {scores.std():.3f}")

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.003058 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 252
[LightGBM] [Info] Number of data points in the train set: 40000, number of used features: 8
[LightGBM] [Info] Start training from score -0.144875
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.006611 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 258
[LightGBM] [Info] Number of data points in the train set: 40000, number of used features: 8
[LightGBM] [Info] Start training from score -0.150000
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.006476 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 256
[LightGBM] [Info] Number of data points in the train set: 4

# XGBoost

In [ ]:
import xgboost as xgb

model = xgb.XGBRegressor(n_estimators=100, random_state=42, n_jobs=-1)
scores = cross_val_score(model, X, y, cv=5, scoring=mae_scorer)

print(f"XGBoost MAE : {-scores.mean():.3f} ± {scores.std():.3f}")

XGBoost MAE : 0.849 ± 0.011


In [ ]:
# Le meilleur c'est LightGBM

# Gradient Boosting

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import cross_val_score

model = GradientBoostingRegressor(n_estimators=100, random_state=42)
scores = cross_val_score(model, X, y, cv=5, scoring=mae_scorer)

print(f"GradientBoosting MAE : {-scores.mean():.3f} ± {scores.std():.3f}")

GradientBoosting MAE : 0.842 ± 0.011


# Modèles avec TimeSeriesSplit

In [ ]:
x_sample = x_train.sample(50000, random_state=42)
x_train_sorted = x_sample.sort_values("date")
X = x_train_sorted[features].fillna(0)
y = y_train.loc[x_train_sorted.index]['p0q0']

# RandomForest

In [ ]:
from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor(n_estimators=20, random_state=42, n_jobs=-1)

In [ ]:
from sklearn.model_selection import TimeSeriesSplit, cross_val_score
tscv = TimeSeriesSplit(n_splits=5)
mae_scorer = make_scorer(mean_absolute_error, greater_is_better=False)

In [ ]:
scores = cross_val_score(model, X, y, cv=tscv, scoring=mae_scorer)
print(f"MAE (TimeSeriesSplit) : {-scores.mean():.3f} ± {scores.std():.3f}")

MAE (TimeSeriesSplit) : 0.956 ± 0.089
